**_WORK IN PROGRESS_**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import datetime as dt

# import sys  
# sys.path.append('/Users/kendra/data_science/Projects/sharedCode')

import util as u

%load_ext autoreload
%autoreload 2

### Load Data

In [15]:
X_train = u.open_pkl('Data/X_train.pkl')
X_test = u.open_pkl('Data/X_test.pkl')
y_train = u.open_pkl('Data/y_train.pkl')
y_test = u.open_pkl('Data/y_test.pkl')

## Tune CatBoost

In [ ]:
cat_cols = ['registered_via']
cat_cols_idx = [list(df_feat1.columns).index(x) for x in cat_cols]

In [ ]:
cb = CatBoostClassifier(cat_features=cat_cols_idx, logging_level='Silent')

## Evaluate model on test data

### Preprocess test data

In [3]:
pca_songs = u.open_pkl('Data/pca_songs.pkl')
pca_trans = u.open_pkl('Data/pca_trans.pkl')

In [16]:
# pca_songs = PCA(n_components=1)
df_songs_pca = X_test[['num_100_mean', 'num_unq_mean', 'num_songs_mean', 'num_songs_sum',
       'total_secs_mean', 'total_secs_sum']]
songs_pca = pca_songs.transform(df_songs_pca)

In [20]:
# pca_trans = PCA(n_components=1)
df_trans_pca = X_test[['payment_plan_days_mode', 'payment_plan_days_sum',
       'plan_list_price_mode', 'plan_list_price_sum',
       'actual_amount_paid_mode', 'actual_amount_paid_sum']]
trans_pca = pca_songs.fit_transform(df_trans_pca)

In [23]:
fs_cols_1 = ['registered_via', 
             'short_pct_mean', 
             'payment_method_most_common_mode',
             'plan_actual_diff_abs_max',
             'is_auto_renew_mode', 'is_cancel_mode',
             'trans_count', 
             'time_since_registration']

df_feat1 = X_test[fs_cols_1]
df_feat1 = df_feat1.join(pd.DataFrame(songs_pca, columns=['song_pca']))
df_feat1 = df_feat1.join(pd.DataFrame(trans_pca, columns=['transactions_pca']))

In [25]:
df_feat1.sample(10)

,registered_via,short_pct_mean,payment_method_most_common_mode,plan_actual_diff_abs_max,is_auto_renew_mode,is_cancel_mode,trans_count,time_since_registration,song_pca,transactions_pca
157611,7.0,0.000000,1.0,0.0,1.0,0.0,1.0,83.43,15762.648464,-67.731168
176760,3.0,0.356986,0.0,0.0,0.0,0.0,1.0,14.30,63624.752520,32.637482
121786,7.0,0.172873,0.0,0.0,1.0,0.0,1.0,81.93,143451.768097,-28.370913
2079,9.0,0.186824,0.0,0.0,1.0,0.0,1.0,138.30,136507.829722,-28.370913
203864,9.0,0.319320,0.0,0.0,1.0,0.0,1.0,151.57,106904.204370,-28.370913
128146,4.0,0.253488,0.0,0.0,0.0,0.0,1.0,4.90,19560.231585,32.637482
8944,9.0,0.480065,0.0,0.0,1.0,0.0,1.0,11.20,19539.710985,-28.370913
213856,7.0,0.258840,1.0,0.0,1.0,0.0,1.0,21.20,27733.155245,-126.771550
116875,3.0,0.129295,0.0,0.0,0.0,0.0,1.0,48.97,51962.985581,3278.645415
81965,4.0,0.383348,0.0,0.0,0.0,0.0,1.0,15.90,58741.666798,-26.402900
